# Семинар к лекции 4 курса Deep Learning. NLP Part 1

На семинаре предлагается рассмотреть этапы преобразования текстов в эмбеддинги с помощью алгоритмов One-Hot, Bag-of-Words, TF-IDF и сравнить методы как между собой, так и с их табличными реализациями.



## Данные и библиотеки


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from collections import defaultdict
import time
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
import pymorphy3
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
import spacy
import pymorphy3
from pymystem3 import Mystem
from collections import defaultdict

В качестве данных для анализа предлагается взять интересный датасет с анекдотами на русском языке.

In [ ]:
!wget https://github.com/Koziev/NLP_Datasets/raw/refs/heads/master/Conversations/Data/extract_dialogues_from_anekdots.tar.xz
!tar -xf extract_dialogues_from_anekdots.tar.xz

In [ ]:
import numpy as np

with open("extract_dialogues_from_anekdots.txt", encoding="utf-8") as file:
    data = file.read()
    data = data.split('\n\n\n\n')
data[69]

'- Анка, ты не знаешь, почему у Петьки волосы на голове такие пышные и кудрявые?\n- А он, Василий Иваныч, их яйцами натирает.\n- Во, акробат!'

In [ ]:
!python -m spacy download ru_core_news_sm
!pip install pymorphy3 pymystem3

## Предобработка

Очень важным этапом при работе с текстами является предобработка входных данных. Здесь и разбиение предложений (и возможно слов) на токены, приведение их к стандартной форме, удаление стоп-слов и лишних символов и прочее.

Для русского языка наиболее популярными являются 4 библиотеки:
  - pymorphy3   -  https://pypi.org/project/pymorphy3/
  - Mystem      -  https://yandex.ru/dev/mystem/
  - SpaCy       -  https://spacy.io/models/ru/
  - natasha     -  https://pypi.org/project/natasha/

Предлагается разобраться с основными методами этих библиотек и сравнить их между собой по качеству и скорости работы.

In [ ]:
morph = pymorphy3.MorphAnalyzer()  # https://pypi.org/project/pymorphy3/

morph.parse('бежит')[0].normal_form

'бежать'

In [ ]:
mystem = Mystem()  # https://yandex.ru/dev/mystem/
mystem.lemmatize('бежит')

['бежать', '\n']

In [ ]:
nlp = spacy.load("ru_core_news_sm", disable=["parser", "ner"])

doc = nlp('бежит')
doc  # https://spacy.io/models/ru/

бежит

In [ ]:
# https://pypi.org/project/natasha/

In [ ]:
def preprocess_text(text):
    cleaned_text = re.sub(r'[^а-яА-ЯёЁ\s0-9S:\[\].,-]', ' ', text)
    tokens = cleaned_text.lower().split()
    tokens = [token for token in tokens if len(token) > 2]

    return tokens

def lemmatize_spacy(tokens):
    nlp = spacy.load("ru_core_news_sm", disable=["parser", "ner"])
    text_to_process = ' '.join(tokens)
    doc = nlp(text_to_process)
    lemmas = [token.lemma_ for token in doc]
    return lemmas

def lemmatize_pymystem(tokens):
    mystem = Mystem()
    text_to_process = ' '.join(tokens)
    lemmas_list = mystem.lemmatize(text_to_process)
    lemmas = [lemma.strip() for lemma in lemmas_list if lemma.strip() and not lemma.isspace()]
    return lemmas

def lemmatize_pymorphy3(tokens):
    morph = pymorphy3.MorphAnalyzer()
    lemmas = []
    for token in tokens:
        parsed = morph.parse(token)[0]
        lemmas.append(parsed.normal_form)

    return lemmas

def compare_lemmatizers(text):
    tokens = preprocess_text(text)
    print(f"Предобработанные токены: {tokens}\n")

    methods = {
        'spaCy': lemmatize_spacy,
        'PyMystem3': lemmatize_pymystem,
        'PyMorphy3': lemmatize_pymorphy3
    }

    results = {}
    times = {}

    for name, func in methods.items():
        start_time = time.time()
        results[name] = func(tokens)
        end_time = time.time()
        times[name] = end_time - start_time
    print("Результаты лемматизации:")
    for name, lemmas in results.items():
        print(f"{name}: {lemmas}")
        print(f"Время выполнения: {times[name]:.4f} секунд\n")

def test_textes(test_text):
    print("Оригинальный текст:", test_text)
    print("\n" + "="*50 + "\n")

    compare_lemmatizers(test_text)

# One-Hot + BoW

На занятии предлагалось разобраться с реализацией tf-idf и дополнив код сравнить с реализацией из библиотеки



Давайте реализуем one-hot:
1. Проходим по всем словам и подсчитываем частоты
2. Если есть ограничение на размер словаря - оставляем топ N
3. Создаем словарь слово->индекс

In [ ]:
def custom_one_hot_encoding(texts, max_features=None):
    """
    Простая реализация One-Hot Encoding для текстовых данных

    Args:
        texts: список текстов для обработки
        max_features: максимальное количество слов для включения в словарь
    """
    # Создаем словарь для подсчета частот слов
    word_freq = defaultdict(int)

    # Разбиваем тексты на слова и подсчитываем частоты
    for text in texts:
        for word in text.lower().split():
            word_freq[word] += 1

    # Сортируем слова по частоте и выбираем top-N
    sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    if max_features:
        sorted_words = sorted_words[:max_features]

    # Создаем словарь слово->индекс
    word_to_index = {word: idx for idx, (word, _) in enumerate(sorted_words)}
    vocab_size = len(word_to_index)

    print(f"Размер словаря: {vocab_size} слов")

    # Создаем one-hot матрицу
    one_hot_matrix = []
    for text in texts:
        # Вектор нулей размером с словарь
        vector = np.zeros(vocab_size)
        for word in text.lower().split():
            if word in word_to_index:
                vector[word_to_index[word]] = 1
        one_hot_matrix.append(vector)

    return np.array(one_hot_matrix), word_to_index

# Применяем нашу реализацию
print("=== Кастомная реализация One-Hot Encoding ===")
start_time = time.time()
custom_encoded, vocab = custom_one_hot_encoding(data[:10000])
custom_time = time.time() - start_time

print(f"Время выполнения: {custom_time:.4f} секунд")
print(f"Размер матрицы: {custom_encoded.shape}")
print(f"Память: {custom_encoded.nbytes} байт")

####################################################

# Используем CountVectorizer с binary=True для one-hot представления
print("\n=== Реализация Scikit-Learn ===")
start_time = time.time()

# CountVectorizer с binary=True создает бинарные признаки (one-hot)
sklearn_vectorizer = CountVectorizer(binary=True)
sklearn_encoded = sklearn_vectorizer.fit_transform(data[:10000])
sklearn_time = time.time() - start_time

print(f"Время выполнения: {sklearn_time:.4f} секунд")
print(f"Размер матрицы: {sklearn_encoded.shape}")
print(f"Память: {sklearn_encoded.data.nbytes} байт (разреженное представление)")
print("Словарь:", sklearn_vectorizer.get_feature_names_out()[:5])

# Сравниваем результаты
print("\n=== Сравнение производительности ===")
comparison = pd.DataFrame({
    'Метод': ['Кастомная реализация', 'Scikit-Learn'],
    'Время (сек)': [custom_time, sklearn_time],
    'Память (байт)': [custom_encoded.nbytes, sklearn_encoded.data.nbytes],
    'Размер матрицы': [str(custom_encoded.shape), str(sklearn_encoded.shape)]
})

print(comparison)

=== Кастомная реализация One-Hot Encoding ===
Размер словаря: 45032 слов
Время выполнения: 3.0682 секунд
Размер матрицы: (10000, 45032)
Память: 3602560000 байт

=== Реализация Scikit-Learn ===
Время выполнения: 0.2848 секунд
Размер матрицы: (10000, 27841)
Память: 1092312 байт (разреженное представление)
Словарь: ['00' '000' '01' '03' '06']

=== Сравнение производительности ===
                  Метод  Время (сек)  Память (байт)  Размер матрицы
0  Кастомная реализация     3.068161     3602560000  (10000, 45032)
1          Scikit-Learn     0.284762        1092312  (10000, 27841)


In [ ]:
sklearn_vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'ящура', 'ёеее', 'ёлочкой'], dtype=object)

In [ ]:
sklearn_encoded

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 102 stored elements and shape (17, 74)>

In [ ]:
#help(CountVectorizer)

## Интересное

In [ ]:
Bag_of_words = CountVectorizer(binary=False) # так по умолчанию

In [ ]:
#help(HashingVectorizer) # для очень больших данных

In [ ]:
# Настройка кодировщика с продвинутыми параметрами
encoder = OneHotEncoder(
    categories='auto',
    drop='if_binary',           # Удаляет только для бинарных признаков
    sparse_output=True,
    handle_unknown='infrequent_if_exist',  # Группировка редких категорий
    min_frequency=1,            # Минимальная частота категории
    max_categories=10,          # Максимальное количество категорий
    feature_name_combiner='concat'  # Формирование имен признаков
)

# tf-idf

На занятии предлагалось разобраться с реализацией tf-idf и дополнив код сравнить с реализацией из библиотеки

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')

class CustomTfidfVectorizer:
    def __init__(self, lowercase=True, remove_stopwords=True, language='russian'):
        self.lowercase = lowercase
        self.remove_stopwords = remove_stopwords
        self.language = language
        self.vocabulary_ = None
        self.idf_ = None
        self.stop_words = set(stopwords.words(language)) if remove_stopwords else set()
        self.stemmer = SnowballStemmer(language)

    def preprocess_text(self, text):
        """Предобработка текста"""
        if self.lowercase:
            text = text.lower()

        # Удаляем знаки препинания и цифры
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\d+', '', text)

        # Токенизация
        tokens = text.split()

        # Удаление стоп-слов и стемминг
        if self.remove_stopwords:
            tokens = [token for token in tokens if token not in self.stop_words]

        tokens = [self.stemmer.stem(token) for token in tokens]

        return tokens

    def fit(self, documents):
        """Обучение модели - построение словаря и вычисление IDF"""
        # Собираем все токены
        all_tokens = []
        doc_token_sets = []

        for doc in documents:
            tokens = self.preprocess_text(doc)
            all_tokens.extend(tokens)
            doc_token_sets.append(set(tokens))

        # Создаем словарь
        self.vocabulary_ = {token: idx for idx, token in enumerate(set(all_tokens))}

        # Вычисляем IDF
        n_docs = len(documents)
        self.idf_ = np.zeros(len(self.vocabulary_))

        for token, idx in self.vocabulary_.items():
            # Количество документов, содержащих токен
            doc_count = sum(1 for doc_tokens in doc_token_sets if token in doc_tokens)
            # Формула IDF с добавлением 1 для избежания деления на 0
            self.idf_[idx] = np.log((n_docs + 1) / (doc_count + 1)) + 1

        return self

    def transform(self, documents):
        """Преобразование документов в TF-IDF матрицу"""
        n_docs = len(documents)
        n_features = len(self.vocabulary_)
        tfidf_matrix = np.zeros((n_docs, n_features))

        for doc_idx, doc in enumerate(documents):
            tokens = self.preprocess_text(doc)

            if not tokens:
                continue

            # Вычисляем TF (термическая частота)
            tf = defaultdict(int)
            total_terms = len(tokens)

            for token in tokens:
                if token in self.vocabulary_:
                    tf[token] += 1

            # Нормализуем TF
            for token in tf:
                tf[token] /= total_terms

            # Вычисляем TF-IDF
            for token, tf_val in tf.items():
                if token in self.vocabulary_:
                    token_idx = self.vocabulary_[token]
                    tfidf_matrix[doc_idx, token_idx] = tf_val * self.idf_[token_idx]

        # L2 нормализация
        norms = np.linalg.norm(tfidf_matrix, axis=1, keepdims=True)
        norms[norms == 0] = 1  # Избегаем деления на 0
        tfidf_matrix = tfidf_matrix / norms

        return tfidf_matrix

    def fit_transform(self, documents):
        """Обучение и преобразование"""
        return self.fit(documents).transform(documents)

# Тестирование и сравнение
def compare_tfidf_implementations():
    # Пример данных на русском языке
    documents = [
        "кот сидит на ковре и смотрит в окно",
        "собака бегает по двору и играет с мячом",
        "кот и собака иногда играют вместе",
        "птица летает высоко в небе над домом",
        "рыба плавает в аквариуме и смотрит на камни"
    ]

    print("Документы для анализа:")
    for i, doc in enumerate(documents, 1):
        print(f"{i}. {doc}")
    print()

    # Наша реализация
    print("=== Наша реализация TF-IDF ===")
    custom_tfidf = CustomTfidfVectorizer()
    custom_matrix = custom_tfidf.fit_transform(documents)

    print("Размерность матрицы:", custom_matrix.shape)
    print("Словарь (первые 10 слов):", dict(list(custom_tfidf.vocabulary_.items())[:10]))
    print("IDF значения (первые 10):", custom_tfidf.idf_[:10])
    print()

    # Реализация sklearn
    print("=== Sklearn TF-IDF ===")
    sklearn_tfidf = TfidfVectorizer(
        lowercase=True,
        stop_words=stopwords.words('russian')
    )
    sklearn_matrix = sklearn_tfidf.fit_transform(documents).toarray()

    print("Размерность матрицы:", sklearn_matrix.shape)
    print("Словарь (первые 10 слов):", dict(list(sklearn_tfidf.vocabulary_.items())[:10]))
    print("IDF значения (первые 10):", sklearn_tfidf.idf_[:10])
    print()

    # Сравнение результатов
    print("=== Сравнение результатов ===")

    # Сравнение матриц
    difference = np.abs(custom_matrix - sklearn_matrix)
    print(f"Средняя разница между матрицами: {np.mean(difference):.6f}")
    print(f"Максимальная разница: {np.max(difference):.6f}")

    # Сравнение косинусного сходства
    print("\nСравнение косинусного сходства:")

    # Для нашей реализации
    custom_similarity = cosine_similarity(custom_matrix)
    print("Косинусное сходство (наша реализация):")
    print(custom_similarity)

    print("\nКосинусное сходство (sklearn):")
    sklearn_similarity = cosine_similarity(sklearn_matrix)
    print(sklearn_similarity)

    # Разница в сходстве
    similarity_diff = np.abs(custom_similarity - sklearn_similarity)
    print(f"\nСредняя разница в косинусном сходстве: {np.mean(similarity_diff):.6f}")

    return custom_tfidf, sklearn_tfidf, custom_matrix, sklearn_matrix

# Дополнительная функция для анализа конкретных слов
def analyze_specific_words(custom_tfidf, sklearn_tfidf, documents):
    """Анализ TF-IDF значений для конкретных слов"""
    print("\n=== Анализ конкретных слов ===")

    test_words = ["кот", "собака", "птица"]

    for word in test_words:
        print(f"\nСлово: '{word}'")

        # В нашей реализации
        if word in custom_tfidf.vocabulary_:
            custom_idx = custom_tfidf.vocabulary_[word]
            print(f"Наша реализация - IDF: {custom_tfidf.idf_[custom_idx]:.4f}")
        else:
            print(f"Наша реализация - слово не найдено в словаре")

        # В sklearn
        if word in sklearn_tfidf.vocabulary_:
            sklearn_idx = sklearn_tfidf.vocabulary_[word]
            print(f"Sklearn - IDF: {sklearn_tfidf.idf_[sklearn_idx]:.4f}")
        else:
            print(f"Sklearn - слово не найдено в словаре")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
custom_tfidf, sklearn_tfidf, custom_matrix, sklearn_matrix = compare_tfidf_implementations()
analyze_specific_words(custom_tfidf, sklearn_tfidf, documents)

# Демонстрация работы с новыми документами
print("\n=== Тест на новых документах ===")
new_docs = [
    "кот играет с собакой в саду",
    "птица сидит на дереве и поет"
]

custom_new = custom_tfidf.transform(new_docs)
sklearn_new = sklearn_tfidf.transform(new_docs).toarray()

print("Наша реализация на новых документах:")
print(custom_new)
print("\nSklearn на новых документах:")
print(sklearn_new)

# NER

Задача NER — это извлечение именованных сущностей из текста (например, имена людей, организации, локации, даты и т.д.)  

Предлагается изучить какими методами её можно решать



In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

nlp = spacy.load("ru_core_news_sm")

text = data[69]

# Обработка текста
doc = nlp(text)

# Извлечение сущностей
for ent in doc.ents:
    print(f"Сущность: {ent.text}, Тип: {ent.label_}, Описание: {spacy.explain(ent.label_)}")

Сущность: России, Тип: LOC, Описание: Non-GPE locations, mountain ranges, bodies of water
Сущность: Антон Силуанов, Тип: PER, Описание: Named person or family.
Сущность: Владимиром Путиным, Тип: PER, Описание: Named person or family.
Сущность: Москве, Тип: LOC, Описание: Non-GPE locations, mountain ranges, bodies of water


In [ ]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

text = data[69]
tokens = text.split()

for token in tokens:
    parsed = morph.parse(token)[0]
    print(f"Слово: {token}, Нормальная форма: {parsed.normal_form}, Часть речи: {parsed.tag.POS}")

# POS

Задача POS — это определение частеречной принадлежности слов в тексте (существительное, глагол, прилагательное и т.д.).

Предлагается изучить какими методами её можно решать

In [ ]:
import spacy

# Загрузка модели для русского языка
nlp = spacy.load("ru_core_news_sm")

text = "Красивая кошка быстро бежала по зеленому полю и ловила мышку"

# Обработка текста
doc = nlp(text)

# POS-разметка
print("POS-разметка с помощью spacy:")
print("-" * 50)
for token in doc:
    print(f"Слово: {token.text:12} POS-тег: {token.pos_:8} Описание: {spacy.explain(token.pos_)}")

POS-разметка с помощью spacy:
--------------------------------------------------
Слово: Красивая     POS-тег: ADJ      Описание: adjective
Слово: кошка        POS-тег: NOUN     Описание: noun
Слово: быстро       POS-тег: ADV      Описание: adverb
Слово: бежала       POS-тег: VERB     Описание: verb
Слово: по           POS-тег: ADP      Описание: adposition
Слово: зеленому     POS-тег: ADJ      Описание: adjective
Слово: полю         POS-тег: NOUN     Описание: noun
Слово: и            POS-тег: CCONJ    Описание: coordinating conjunction
Слово: ловила       POS-тег: VERB     Описание: verb
Слово: мышку        POS-тег: NOUN     Описание: noun
Слово: .            POS-тег: PUNCT    Описание: punctuation


In [ ]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

text = "Красивая кошка быстро бежала по зеленому полю"
tokens = text.split()

print("POS-разметка с помощью pymorphy3:")
print("-" * 50)
for token in tokens:
    parsed = morph.parse(token)[0]  # Берем наиболее вероятный разбор
    pos_tag = parsed.tag.POS
    full_grammar = parsed.tag
    normal_form = parsed.normal_form

    print(f"Слово: {token:12} POS: {pos_tag:8} Нормальная форма: {normal_form:12} Грамматика: {full_grammar}")

POS-разметка с помощью pymorphy3:
--------------------------------------------------
Слово: Красивая     POS: ADJF     Нормальная форма: красивый     Грамматика: ADJF,Qual femn,sing,nomn
Слово: кошка        POS: NOUN     Нормальная форма: кошка        Грамматика: NOUN,anim,femn sing,nomn
Слово: быстро       POS: ADVB     Нормальная форма: быстро       Грамматика: ADVB
Слово: бежала       POS: VERB     Нормальная форма: бежать       Грамматика: VERB,perf,intr femn,sing,past,indc
Слово: по           POS: PREP     Нормальная форма: по           Грамматика: PREP
Слово: зеленому     POS: ADJF     Нормальная форма: зелёный      Грамматика: ADJF,Qual masc,sing,datv
Слово: полю         POS: NOUN     Нормальная форма: поле         Грамматика: NOUN,inan,neut sing,datv


# Классификация (регулярки)

In [ ]:
# найдите все анекдоты про Вовочку в датасете
# предлагается рассмотреть два варианта
# - через регулярные выражения (не стоит забывать об этом мощном способе)
# - через поиск эмбеддинга к слову "Вовочка"